In [4]:
!pip install langchain
!pip install langchain_core
!pip install langchain_community
!pip install chromadb
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.1/467.1 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.11
    Uninstalling langchain-text-splitters-0.3.11:
      Successfully uninstalled langchain-text-splitters-0.3.11
ERROR: pip's dependency resolver

In [18]:
!pip install chromadb

In [5]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 95.5 MB/s eta 0:00:00


# Data Ingestion

In [4]:
from langchain_core.documents import Document
from langchain_community.document_loaders import DirectoryLoader

In [5]:
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader

dir_loader = DirectoryLoader(
    "../content",
    glob="**/*.pdf" , #pattern to match the file
    loader_cls= PyMuPDFLoader,
    show_progress=True
)

pdf = dir_loader.load()
print(pdf)

100%|██████████| 1/1 [00:00<00:00,  2.46it/s]

[Document(metadata={'producer': 'iLovePDF', 'creator': '', 'creationdate': '', 'source': '../content/RAG_interveiw_question.pdf', 'file_path': '../content/RAG_interveiw_question.pdf', 'total_pages': 42, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-10-12T05:46:27+00:00', 'trapped': '', 'modDate': 'D:20251012054627Z', 'creationDate': '', 'page': 0}, page_content=''), Document(metadata={'producer': 'iLovePDF', 'creator': '', 'creationdate': '', 'source': '../content/RAG_interveiw_question.pdf', 'file_path': '../content/RAG_interveiw_question.pdf', 'total_pages': 42, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-10-12T05:46:27+00:00', 'trapped': '', 'modDate': 'D:20251012054627Z', 'creationDate': '', 'page': 1}, page_content='🚀AIxFunda Newsletter                                                          aixfunda.substack.com                         \n \nQ1. Explain the requirement of RAG whe

# Embedding and VectorDB

In [6]:
import chromadb
from chromadb.config import Settings
import uuid
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
from typing import List,Any
import numpy as np
from sentence_transformers import SentenceTransformer

class EmbeddingManager:
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"✅ Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"❌ Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        if not self.model:
            raise ValueError("Model not loaded")

        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


In [8]:
embedding_manager = EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ Model loaded successfully. Embedding dimension: 384


In [9]:
### vecotr store

In [10]:
import os
import chromadb
from chromadb.config import Settings
class VectorStore:
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../content/vectorstore"):
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)

            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")

        print(f"Adding {len(documents)} documents to vector store...")

        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []

        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)

            # Document content
            documents_text.append(doc.page_content)

            # Embedding
            embeddings_list.append(embedding.tolist())

        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise


In [11]:
vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 84


In [12]:
# convert text to embeddings

In [13]:
import uuid
from sklearn.metrics.pairwise import cosine_similarity

text = [doc.page_content for doc in pdf]
embeddings = embedding_manager.generate_embeddings(text)

## stor into vector database
vectorstore.add_documents(pdf,embeddings)

## generate embeddings
embeddings = embedding_manager.generate_embeddings(text)

## stor into vector database
vectorstore.add_documents(pdf,embeddings)

Generating embeddings for 42 texts...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Generated embeddings with shape: (42, 384)
Adding 42 documents to vector store...
Successfully added 42 documents to vector store
Total documents in collection: 126
Generating embeddings for 42 texts...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Generated embeddings with shape: (42, 384)
Adding 42 documents to vector store...
Successfully added 42 documents to vector store
Total documents in collection: 168


In [14]:
# Retriver Pipeline form vector store

In [15]:
from typing import List, Dict, Any, Tuple
class RAGRetriever:
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")

        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]

        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )

            # Process results
            retrieved_docs = []

            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]

                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance

                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })

                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")

            return retrieved_docs

        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

In [27]:
rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [17]:
rag_retriever.retrieve('requirement of RAG when LLMs are already powerful')

Retrieving documents for query: 'requirement of RAG when LLMs are already powerful'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_f544ef77_1',
  'content': '🚀AIxFunda Newsletter                                                          aixfunda.substack.com                         \n \nQ1. Explain the requirement of RAG when LLMs are already powerful. \n \nLLMs are powerful, as they are trained on large volumes of data using sophisticated \ntechniques. However, LLMs because of knowledge cutoff (static knowledge), struggle to \nanswer queries related to the latest events or the data not present in their training \ncorpus.  \n \nRAG addresses this challenge by retrieving relevant context from external knowledge \nsources, which allows LLMs to provide accurate responses. This is why RAG is essential \nfor LLM-based applications that need to be accurate. Otherwise, LLMs alone might \nprovide you answers that are incomplete or outdated. \n \nQ2. Is RAG still relevant in the era of long context LLMs? \n \nRAG is still important even with long context LLMs. This is because long-context LLMs \nwithout RAG hav

In [18]:
# Integeration the context with groqapi through the LLM

In [19]:
GROQ_API_KEY = "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"

In [20]:
from dotenv import load_dotenv
load_dotenv()

True

In [21]:
%%writefile .env
GROQ_API_KEY = "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"

Overwriting .env


In [22]:
!pip install langchain-groq


In [23]:
!pip install python-dotenv

In [24]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant",temperature=0.1,max_tokens=1024)


def rag_simple(query,retriever,llm,top_k=3):
    ## retriever the context
    results=retriever.retrieve(query,top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."

    prompt=f"""Use the following context to answer the question concisely.
        Context: {context}

        Question: {query}

        Answer:"""

    response=llm.invoke([prompt.format(context=context,query=query)])
    return response.content

In [25]:
answer=rag_simple("What is the requirement of RAG when LLMs are already powerful?",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'What is the requirement of RAG when LLMs are already powerful?'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
The requirement of RAG when LLMs are already powerful is to address the challenge of knowledge cutoff (static knowledge). LLMs struggle to answer queries related to the latest events or data not present in their training corpus. RAG retrieves relevant context from external knowledge sources, allowing LLMs to provide accurate responses.


In [26]:
# Enchance Rag pipeline

In [30]:
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}

    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])

    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])

    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("need for rag when LLM exists", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:1000])

Retrieving documents for query: 'need for rag when LLM exists'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
Answer: RAG (Retrieval-Augmented Generation) is still essential even when LLMs (Large Language Models) are powerful because LLMs have a knowledge cutoff (static knowledge) and struggle to answer queries related to the latest events or data not present in their training corpus. RAG addresses this challenge by retrieving relevant context from external knowledge sources, allowing LLMs to provide accurate responses.
Sources: [{'source': '../content/RAG_interveiw_question.pdf', 'page': 1, 'score': 0.2769176959991455, 'preview': '🚀AIxFunda Newsletter                                                          aixfunda.substack.com                         \n \nQ1. Explain the requirement of RAG when LLMs are already powerful. \n \nLLMs are powerful, as they are trained on large volumes of data using sophisticated \ntechniques. Howeve...'}, {'source': '../content/RAG_interveiw_question.pdf', 'page': 1, 'score': 0.27

In [32]:
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("what is rag and llm and why they needed", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

Retrieving documents for query: 'what is rag and llm and why they needed'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
Streaming answer:
Use the following context to answer the question concisely.
Context:
🚀AIxFunda Newsletter                                                          aixfunda.substack.com                         
 
Q1. Explain the requirement of RAG when LLMs are already powerful. 
 
LLMs are powerful, as they are trained on large volumes of data using sophisticated 
techniques. However, LLMs because of knowledge cutoff (static knowledge), struggle to 
answer queries related to the latest events or the data not present in their training 
corpus.  
 
RAG addresses this challenge by retrieving relevant context from external knowledge 
sources, which allows LLMs to provide accurate responses. This is why RAG is essential 
for LLM-based applications that need to be accurate. Otherwise, LLMs alone might 
provide you answers that are incomplete or outdated. 
 
Q2. Is RAG still relevant in the era of long context